# CAPM EUA

In [ ]:
from scipy import stats
import pandas_datareader.data as web
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt

%matplotlib inline

ds_yahoo = 'yahoo'

last_year = datetime.datetime.now().year - 1
start = datetime.datetime.now().replace(year=last_year)
end = datetime.datetime.now()

### S&P500 ETF

In [ ]:
spy = web.DataReader('SPY', ds_yahoo, start, end)

### Carteira de Ações

In [ ]:
tickers = ['LB', 'M']

lb = web.DataReader(tickers[0], ds_yahoo, start, end)
m = web.DataReader(tickers[1], ds_yahoo, start, end)

### Acumulado

In [ ]:
spy['Acumulado'] = spy['Adj Close'] / spy['Adj Close'].iloc[0]
lb['Acumulado'] = lb['Adj Close'] / lb['Adj Close'].iloc[0]
m['Acumulado'] = m['Adj Close'] / m['Adj Close'].iloc[0]

In [ ]:
spy['Acumulado'].plot(label='spy', figsize=(17,6))
lb['Acumulado'].plot(label='Limited Brands')
m['Acumulado'].plot(label='Macys')
plt.legend();

# Calculando o Retorno Diário de cada ação

In [ ]:
spy['Retorno Diário'] = spy['Adj Close'].pct_change(1)
lb['Retorno Diário'] = lb['Adj Close'].pct_change(1)
m['Retorno Diário'] = m['Adj Close'].pct_change(1)

# Calculando o Retorno Diário de toda a Carteira

### Normalizando o retorno diário de cada ativo do portfólio

In [ ]:
for acao_df in (lb, m):
    acao_df['Retorno Normalizado'] = acao_df['Adj Close'] / acao_df.iloc[0]['Adj Close']

### Alocando cada ativo com um peso específico

In [ ]:
# Pesos atuais
pesos = [.5, .5]

for acao_df, alocacao in zip((lb, m), pesos):
    acao_df['Alocacao'] = acao_df['Retorno Normalizado'] * alocacao

### Posição de cada ativo no portfólio

In [ ]:
for acao_df in (lb, m):
    acao_df['Posicao'] = acao_df['Alocacao'] * 70000

In [ ]:
todas_posicoes = [lb['Posicao'], m['Posicao']]

# DF que terá colunas adicionadas como Total e Retorno Diário
portfolio_posicao = pd.concat(todas_posicoes, axis=1)

In [ ]:
portfolio_posicao.columns = ['LB', 'M']

In [ ]:
portfolio_posicao['Total'] = portfolio_posicao.sum(axis=1)

In [ ]:
portfolio_posicao['Retorno Diário'] = portfolio_posicao['Total'].pct_change(1)

In [ ]:
portfolio_posicao.head()

### Carteira EUA x S&P500

In [ ]:
plt.scatter(portfolio_posicao['Retorno Diário'], spy['Retorno Diário'], alpha=0.5);

In [ ]:
beta_p, alpha_p, r_value_p, p_value_p, std_err_p = stats.linregress(portfolio_posicao['Retorno Diário'].iloc[1:],
                                                                    spy['Retorno Diário'].iloc[1:])

In [ ]:
print('beta: {}'.format(beta_p))
print('alpha: {}'.format(alpha_p))
print('r_value: {}'.format(r_value_p))
print('p_value: {}'.format(p_value_p))
print('std_err: {}'.format(std_err_p))

### Limited Brands x S&P500

In [ ]:
plt.scatter(lb['Retorno Diário'], spy['Retorno Diário'], alpha=0.5);

In [ ]:
beta_i, alpha_i, r_value_i, p_value_i, std_err_i = stats.linregress(lb['Retorno Diário'].iloc[1:],
                                                                    spy['Retorno Diário'].iloc[1:])

In [ ]:
print('beta: {}'.format(beta_i))
print('alpha: {}'.format(alpha_i))
print('r_value: {}'.format(r_value_i))
print('p_value: {}'.format(p_value_i))
print('std_err: {}'.format(std_err_i))

### Macy's x S&P500

In [ ]:
plt.scatter(m['Retorno Diário'], spy['Retorno Diário'], alpha=0.5);

In [ ]:
beta_f, alpha_f, r_value_f, p_value_f, std_err_f = stats.linregress(m['Retorno Diário'].iloc[1:],
                                                                    spy['Retorno Diário'].iloc[1:])

In [ ]:
print('beta: {}'.format(beta_f))
print('alpha: {}'.format(alpha_f))
print('r_value: {}'.format(r_value_f))
print('p_value: {}'.format(p_value_f))
print('std_err: {}'.format(std_err_f))